# Preparing Collected Data

In this notebook we'll go through the process of preparing the collected article data. The steps outlined here will allow for reproducing the results in production. The main interest here is to normalize the data in such a way that it can be easily utilized in exploration and modeling in production.

## Imports

These are all the modules that we'll need to run the code in this notebook.

In [45]:
import numpy as np
import pandas as pd

import unicodedata
import re
import datetime

## Pulling the Training Data

A set of article data has been set aside for the purposes of data analysis and model training. Here I'll load that data and take a look at what features we're working with.

In [2]:
df = pd.read_csv('../data/articles-with-topic-label.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_id         250 non-null    object 
 1   publication_id  250 non-null    object 
 2   title           250 non-null    object 
 3   subtitle        248 non-null    object 
 4   date            250 non-null    object 
 5   word_count      250 non-null    float64
 6   read_time       250 non-null    float64
 7   url             250 non-null    object 
 8   tags            250 non-null    object 
 9   topics          250 non-null    object 
 10  lang            250 non-null    object 
 11  author          244 non-null    object 
 12  publication     245 non-null    object 
dtypes: float64(2), object(11)
memory usage: 25.5+ KB


## Removing Null Values

The first that needs to be done is removing null observations that won't provide any value to us. Anything missing in the author and publication columns won't be of any use to us since these articles are not visible publicly. Details about this issue can be found in the building_labeled_data.ipynb notebook. Here we'll simply remove these rows. Null values in the subtitle column are not a problem for us since this is normal for some articles, but we'll probably want to change these to have empty strings instead of np.NaN.

In [4]:
# Here we're filling nulls in the subtitle column with an empty string.
df[['subtitle']] = df[['subtitle']].fillna('')

In [5]:
# Here we're removing all remaining rows with missing values.
df = df.dropna()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 0 to 249
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_id         241 non-null    object 
 1   publication_id  241 non-null    object 
 2   title           241 non-null    object 
 3   subtitle        241 non-null    object 
 4   date            241 non-null    object 
 5   word_count      241 non-null    float64
 6   read_time       241 non-null    float64
 7   url             241 non-null    object 
 8   tags            241 non-null    object 
 9   topics          241 non-null    object 
 10  lang            241 non-null    object 
 11  author          241 non-null    object 
 12  publication     241 non-null    object 
dtypes: float64(2), object(11)
memory usage: 26.4+ KB


## Title and Subtitle

The titles and subtitles of each article will be heavily used for the purposes of both determining the topic(s) of an article and determining whether a user might find the article interesting to read. For these columns we will likely want to perform some normalization by converting everything to lowercase. We'll also want to remove any special characters and punctuation that exist in the strings. We want to keep only words and numbers as these are the only things that will provide value to us.

In [7]:
df[['title', 'subtitle']].head()

,title,subtitle
0,Top 10 In-Demand programming languages to lear...,"Python, Java, JavaScript, C, C#, C++, Swift, P..."
1,Cheat Sheet: Importing and exporting all file ...,Hi everyone! In this post we’ll review how to ...
2,Python Packages That Apple Uses,Welcome back! Python is an awesome programming...
3,An Easy Introduction to NumPy Arrays,"What, How, and Why."
4,Alternative Method to Choose the Right Machine...,"In this article, I will introduce you to a spe..."


In [12]:
# Convert everything to lowercase.
df['title'] = df['title'].apply(str.lower)
df['subtitle'] = df['subtitle'].apply(str.lower)

In [13]:
df[['title', 'subtitle']].head()

,title,subtitle
0,top 10 in-demand programming languages to lear...,"python, java, javascript, c, c#, c++, swift, p..."
1,cheat sheet: importing and exporting all file ...,hi everyone! in this post we’ll review how to ...
2,python packages that apple uses,welcome back! python is an awesome programming...
3,an easy introduction to numpy arrays,"what, how, and why."
4,alternative method to choose the right machine...,"in this article, i will introduce you to a spe..."


In [17]:
# Normalize all characters to utf-8.
df['title'] = df['title'].apply(
    lambda column: unicodedata.normalize('NFKD', column).encode('ascii', 'ignore').decode('utf-8', 'ignore')
)
df['subtitle'] = df['subtitle'].apply(
    lambda column: unicodedata.normalize('NFKD', column).encode('ascii', 'ignore').decode('utf-8', 'ignore')
)

In [18]:
df[['title', 'subtitle']].head()

,title,subtitle
0,top 10 in-demand programming languages to lear...,"python, java, javascript, c, c#, c++, swift, p..."
1,cheat sheet: importing and exporting all file ...,hi everyone! in this post well review how to i...
2,python packages that apple uses,welcome back! python is an awesome programming...
3,an easy introduction to numpy arrays,"what, how, and why."
4,alternative method to choose the right machine...,"in this article, i will introduce you to a spe..."


In [21]:
df.head(1).title.values[0]

'top 10 in-demand programming languages to learn in 2022'

In [22]:
df.head(1).subtitle.values[0]

'python, java, javascript, c, c#, c++, swift, php, go, rust'

We want to remove punctuation, but we don't want to remove all special characters. For instance, in "c++" and "c#" we would want to keep the '+' and '#' characters since these provide value to us. Additionally, something like "in-demand" should be converted to "in demand". This part of the preparation is very likely to evolve as more special cases are discovered.

In [25]:
df['title'] = (
    df['title']
    .apply(lambda column: column.replace('-', ' '))
    .apply(lambda column: re.sub(r'[^a-z0-9\s\+\#]', '', column))
)
df['subtitle'] = (
    df['subtitle']
    .apply(lambda column: column.replace('-', ' '))
    .apply(lambda column: re.sub(r'[^a-z0-9\s\+\#]', '', column))
)

In [26]:
df[['title', 'subtitle']].head()

,title,subtitle
0,top 10 in demand programming languages to lear...,python java javascript c c# c++ swift php go rust
1,cheat sheet importing and exporting all file t...,hi everyone in this post well review how to im...
2,python packages that apple uses,welcome back python is an awesome programming ...
3,an easy introduction to numpy arrays,what how and why
4,alternative method to choose the right machine...,in this article i will introduce you to a spec...


## Date

How the date is prepared will depend on the source of the information. Different sources provide the date in different formats. The format for the date in the training data is roughly the desired format of the date, but we don't need the time information so that can be removed.

In [28]:
df[['date']].head()

,date
0,2022-07-13 21:46:25
1,2022-07-13 21:39:33
2,2022-07-13 21:22:54
3,2022-07-13 20:51:17
4,2022-07-13 19:59:23


In [33]:
# This will convert the dates into the desired format.
pd.to_datetime(df['date']).dt.date

0      2022-07-13
1      2022-07-13
2      2022-07-13
3      2022-07-13
4      2022-07-13
          ...    
245    2022-07-13
246    2022-07-13
247    2022-07-13
248    2022-07-13
249    2022-07-13
Name: date, Length: 241, dtype: object

In [43]:
# Here I'll create a dataframe containing the various date formats I've seen so far to see if the above method
# will work for mixed date formats.

dummy_df = pd.DataFrame({'date' : ['2022-07-13 21:22:54', '2022-07-11T20:03:23', 'Jul 6, 2022']})
dummy_df

,date
0,2022-07-13 21:22:54
1,2022-07-11T20:03:23
2,"Jul 6, 2022"


In [44]:
pd.to_datetime(dummy_df['date']).dt.date

0    2022-07-13
1    2022-07-11
2    2022-07-06
Name: date, dtype: object

This strategy works, but with a caveat. This date "2022-07-11T20:03:23" is originally "2022-07-11T20:03:23Z" and this date "Jul 6, 2022" is originally "Jul 6". This means that depending on the format there would need to be slight modifications to the date string.

In [58]:
# Here I'll make a function that will look at the date string and make a modification based on the format.
def modify_date_string(date):
    if date.endswith('Z'):
        return date.replace('Z', '')
    elif re.match(r'[a-zA-Z]{3}\s[0-9]', date):
        return date + ', ' + str(datetime.datetime.now().year)
    else:
        return date

In [59]:
dummy_df = pd.DataFrame({'date' : ['2022-07-13 21:22:54', '2022-07-11T20:03:23Z', 'Jul 6']})
dummy_df

,date
0,2022-07-13 21:22:54
1,2022-07-11T20:03:23Z
2,Jul 6


In [60]:
dummy_df['date'] = dummy_df['date'].apply(modify_date_string)
dummy_df

,date
0,2022-07-13 21:22:54
1,2022-07-11T20:03:23
2,"Jul 6, 2022"


Now with the string modifications applied and the date format conversion above we'll be able to properly format the dates.

## Word Count and Read Time

These two features will be used to determine how to fill in read_time when a read time is not provided. The only change that needs to be done here is to cast these to int types.

In [63]:
df[['word_count', 'read_time']].head()

,word_count,read_time
0,4208.0,21.0
1,47.0,0.0
2,572.0,2.0
3,866.0,3.0
4,1421.0,6.0


In [64]:
df[['word_count', 'read_time']] = df[['word_count', 'read_time']].astype(int)
df[['word_count', 'read_time']].head()

,word_count,read_time
0,4208,21
1,47,0
2,572,2
3,866,3
4,1421,6


## Tags and Topics

In [65]:
df[['tags', 'topics']].head()

,tags,topics
0,"['programming', 'software-development', 'pytho...",{}
1,"['python', 'data', 'data-science', 'sql', 'pan...",['programming']
2,"['python', 'programming', 'coding', 'software-...",['programming']
3,"['numpy', 'arrays', 'python', 'data-science', ...","['data-science', 'programming']"
4,"['machine-learning', 'python', 'computer-scien...",['machine-learning']
